In [ ]:
from pathlib import Path
import logging
from dotenv import dotenv_values
import pandas as pd
import geopandas as gpd
import numpy as np

from network_lumping import run_network_lumping_to_generate_basins
from shapely.geometry import LineString, Polygon

logging.basicConfig(level=logging.DEBUG)

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
config = dotenv_values("..\\.env")
base_dir = Path(config["DATA_DIR"])

case_name = "test"
# case_name = "aa_en_maas"

In [ ]:
case_path = Path(base_dir, case_name)

In [ ]:
network = run_network_lumping_to_generate_basins(
    path=case_path, 
    html_file_name=None
)

In [ ]:
network.folium_map

In [ ]:
## TEST DETECT  ##
uitstroom_nodes = network.uitstroom_punten.representatieve_node.values
direction = network.direction
 
uitstroom_edges = None

for node in network.uitstroom_nodes.nodeID.values:
    upstream_edges = network.uitstroom_edges[(network.uitstroom_edges.node_start==node)].copy()
 
    uitstroom_punten_columns = [f"{direction}_node_{n}" for n in uitstroom_nodes]
 
    # drop if all columns are False
    upstream_edges[uitstroom_punten_columns] = upstream_edges[uitstroom_punten_columns].replace(False, np.nan).copy()
    upstream_edges = upstream_edges.dropna(subset=uitstroom_punten_columns, how='all')
 
    # checked if all columns are equal
    edges = upstream_edges.drop_duplicates(subset=uitstroom_punten_columns)
    if len(edges) > 1:
        if uitstroom_edges is None:
            uitstroom_edges = edges.copy()
        else:
            uitstroom_edges = pd.concat([uitstroom_edges, edges])
   
uitstroom_edges[uitstroom_punten_columns] = uitstroom_edges[uitstroom_punten_columns].replace(np.nan, False).copy()